In [121]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
from enum import Enum
import json
import scipy.sparse as sps
import scipy.sparse.linalg as spsla
import trimesh
import jax.numpy as jnp
import DECMesh

In [122]:
# import sympy as sp
# import numpy as np
# from sympy import Rational as Frac

# # Define the symbolic chart coordinate variables
# a, b = sp.symbols("u v")

# # This represents the parametrization of the paraboloid surface
# f = sp.Matrix([a, a**2 + b**2, b])
# # f = sp.Matrix([a, b * sp.sin(a * sp.pi * 1.3), b * sp.cos(a * sp.pi * 1.3)])
# # f = sp.Matrix([u, Frac(4, 5) * sp.exp(-5 * (u * u + v * v)), v])

# # Define the scalar function phi(u, v).
# # It uses the local coordinate system of the manifold to assign scalar values to it.
# # You can replace phi with any scalar function defined on the manifold
# phi = a ** 3 + 3 * b**2

In [123]:
# ### Obsolete sympy version

# f_numpy = sp.lambdify((a, b), f, "numpy")
# phi_numpy = sp.lambdify((a, b), phi, "numpy")

# # Compute the partial derivatives of f with respect to x and y
# # These are the tangent vectors to the surface
# f_x = f.diff(a)  # Partial derivative with respect to x
# f_y = f.diff(b)  # Partial derivative with respect to y

# # Compute the metric tensor g_ij = partial_i f . partial_j f
# # The metric tensor is a 2x2 matrix
# g_11 = f_x.dot(f_x)  # g_11 = <f_x, f_x>
# g_12 = f_x.dot(f_y)  # g_12 = <f_x, f_y>
# g_21 = f_y.dot(f_x)  # g_21 = <f_y, f_x>
# g_22 = f_y.dot(f_y)  # g_22 = <f_y, f_y>

# # Assemble the metric tensor as a matrix
# g = sp.Matrix([[g_11, g_12], [g_21, g_22]])

# # Compute the determinant of the metric tensor |g|
# g_det = g.det()

# # Compute the inverse metric tensor g^{ij}
# g_inv = g.inv()

# # Compute the partial derivatives of phi with respect to x and y
# phi_x = phi.diff(a)
# phi_y = phi.diff(b)

# # Assemble the gradient vector of phi
# grad_phi = sp.Matrix([phi_x, phi_y])

# # Compute the components of the Laplace-Beltrami operator
# # Delta_phi = (1 / sqrt(|g|)) * d/dx_i [ sqrt(|g|) * g^{ij} * d_phi/dx_j ]
# # We'll compute each term step by step

# # First, compute sqrt(|g|)
# sqrt_g_det = sp.sqrt(g_det)

# # Multiply sqrt(|g|) with the inverse metric tensor and gradient of phi
# # This gives us the components inside the divergence operator
# divergence_components = sqrt_g_det * (g_inv * grad_phi)

# display(divergence_components)
# gradient_func = sp.lambdify((a,b), divergence_components, "numpy")

# print(gradient_func(2.0, 2.0))

# # Now, compute the divergence by taking the partial derivatives
# divergence_x = sp.diff(divergence_components[0], a)
# divergence_y = sp.diff(divergence_components[1], b)

# # Sum the partial derivatives to get the divergence
# divergence = divergence_x + divergence_y

# # Finally, compute the Laplace-Beltrami operator
# laplace_beltrami_phi = (1 / sqrt_g_det) * divergence

# # Simplify the expression for clarity
# laplace_beltrami_phi = sp.simplify(laplace_beltrami_phi)

# # Now, vectorize the function for numerical computation using NumPy
# # We'll create a lambda function that can take NumPy arrays as input

# display(laplace_beltrami_phi)

# # Convert the symbolic expression to a numerical function
# laplace_beltrami_func = sp.lambdify((a,b), laplace_beltrami_phi, "numpy")


# # Define a function that takes an (n, 2) array of points and returns an (n,) array of curvature scalars
# def compute_laplace_beltrami(points):
#     """
#     Compute the Laplace-Beltrami operator of phi at given points on the chart.

#     Parameters:
#     points (numpy.ndarray): An (n, 2) array where each row is a point [x, y].

#     Returns:
#     numpy.ndarray: An (n,) array of Laplace-Beltrami operator values at the given points.
#     """
#     x_vals = points[:, 0]
#     y_vals = points[:, 2]
#     # Evaluate the Laplace-Beltrami operator at the given points
#     lb_values = laplace_beltrami_func(x_vals, y_vals)
#     return lb_values

In [124]:
import jax
import jax.numpy as jnp


# def f(uv):
#     """helix"""
#     u, v = uv
#     # return jnp.array([u, u**2 + v**2, v])
#     u_ = jnp.atan(u * jnp.pi / 2)

#     helix = jnp.array([u_, v * jnp.sin(u * jnp.pi), v * jnp.cos(u * jnp.pi)])

#     r = helix[1] + 1.5
#     theta = helix[0] * jnp.pi / 2

#     return jnp.array([r * jnp.cos(theta), r * jnp.sin(theta), helix[2]])


# def f(uv):
#     """sort-of-disc"""
#     u, v = uv

#     dist = (jnp.sqrt(u**2 + v**2))

#     scale = (dist+0.6)/(dist**2+0.6)  # lower 0.6 to make it more disc-like

#     return jnp.array([u*scale, 0, v*scale])


def f(uv):
    """hyperbolic"""
    u, v = uv

    dist = jnp.sqrt(u**2 + v**2)
    input_angle = jnp.arctan2(v, u)

    # given angle and distance I need to return 2D point, expressed in polar coordinates, and then convert to cartesian
    cone = jnp.sin(input_angle)

    input_angle += jnp.sin(dist * 3.0) * 0.4

    x = dist * jnp.cos(input_angle)
    z = dist * jnp.sin(input_angle)

    cone = jnp.sin(input_angle * dist * 3.0) * dist * 0.1

    return jnp.array([x, cone, z])


def phi(uv):
    u, v = uv
    return jnp.sin(u * 3.2 * jnp.pi + v * jnp.pi * 2.3)


def lap(uv):
    def psi(ab):
        grad_phi = jax.grad(phi)(ab)
        Jf = jax.jacfwd(f)(ab)
        g = jnp.dot(Jf.T, Jf)
        sqrt_g_det = jnp.sqrt(jnp.linalg.det(g))

        # gx = grad_phi <=> x = g_inv * grad_phi
        g_inv_times_grad_phi = jnp.linalg.solve(g, grad_phi)  #!

        return sqrt_g_det * g_inv_times_grad_phi

    Jf = jax.jacfwd(f)(uv)
    g = jnp.dot(Jf.T, Jf)
    sqrt_g_det = jnp.sqrt(jnp.linalg.det(g))

    jac_psi = jax.jacfwd(psi)(uv)

    return (1 / sqrt_g_det) * jnp.sum(jnp.diag(jac_psi))


analytical_laplace_beltrami_func = jax.jit(jax.vmap(lap))
f_func = jax.jit(jax.vmap(f))
phi_func = jax.jit(jax.vmap(phi))

In [125]:
import scipy.sparse as sparse
import scipy.sparse.linalg as spla


def modify_poisson_system(L, f, boundary_nodes, boundary_values):
    L = L.tocsr()  # Convert L to CSR format for efficient row operations
    f_modified = f.copy()

    for i in boundary_nodes:
        # Zero out the i-th row in L
        L.data[L.indptr[i] : L.indptr[i + 1]] = 0
        # Set the diagonal element to 1
        L[i, i] = 1
        # Set the corresponding entry in f to the boundary value

    return L, f_modified


def solve_poisson_equation(L, source, boundary_nodes, boundary_values):
    source = source.copy()
    source[boundary_nodes] = boundary_values[boundary_nodes]
    L_modified = matrix_to_identity_at_boundary(L, boundary_nodes)
    print("rank of L:", np.linalg.matrix_rank(L_modified.toarray()))
    print("shape of L:", L_modified.shape)
    # L and f have been modified so the solution will ensure the boundary conditions are satisfied
    u = spla.spsolve(L_modified, source)

    return u


### HEAT EQUATION


def solve_heat_equation(
    initial_condition, L, h, boundary_nodes, boundary_values, f, num_steps
):
    n = len(initial_condition)
    u_n = initial_condition.copy()
    u_n[boundary_nodes] = boundary_values[boundary_nodes]

    # Modify the system matrix once (since boundary conditions are time-independent)
    I = sparse.identity(n, format="csr")  # Identity matrix
    A = I - h * L  # System matrix for Backward Euler
    # Convert A to CSR format for efficient row operations
    A = A.tocsr()

    # Precompute h * f if f is time-independent
    hf = h * f

    steps = [u_n.copy()]
    for _ in range(num_steps - 1):
        # Construct right-hand side
        b = u_n.copy()

        # For interior nodes, add source term
        b[~boundary_nodes] += hf[~boundary_nodes]

        # For boundary nodes, enforce boundary conditions
        b[boundary_nodes] = boundary_values[boundary_nodes]

        # Solve for the next time step
        u_next = spla.spsolve(A, b)

        steps.append(u_next.copy())
        # Update the solution
        u_n = u_next

    return steps


def matrix_to_identity_at_boundary(M, boundary_nodes):
    # Modify rows corresponding to boundary nodes
    for i, is_boundary in enumerate(boundary_nodes):
        if not is_boundary:
            continue
        # Zero out the i-th row in A
        M.data[M.indptr[i] : M.indptr[i + 1]] = 0
        # Set the diagonal element to 1
        M[i, i] = 1
    return M

In [126]:
mesh = Mesh()
boundary_values = np.zeros(len(mesh.vertices))
boundary_values[mesh.boundary_mask] = -1
boundary_values[:24] = 1
boundary_values[-25:] = 1

border_size = mesh.boundary_mask.sum()
increasing = np.linspace(0, 1, border_size)
alternating = (
    ([-1, -1, -1, 1, 1, 1] * 100)[:25]
    + ([-1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1] * 100)[:25]
    + ([-1, -1, -1, 1, 1, 1] * 100)[:25]
)

boundary_values[mesh.boundary_mask] = alternating

/var/folders/vc/1d1z34l176gfm4swwfttvslr0000gn/T/ipykernel_33307/3274498193.py:58: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  -spsla.spsolve(self.star0, self.d0.T @ self.star1_circ @ self.d0)


In [127]:
# mesh = Mesh()

# L_sps = sps.csr_matrix(mesh.laplace_matrix)
# f = np.zeros(len(mesh.vertices))
# a = solve_poisson_equation(L_sps, f, mesh.boundary_vertices_mask, boundary_values)
# u_s = solve_heat_equation(
#     initial_condition=u0,
#     L=L_sps,
#     h=0.02,
#     boundary_nodes=mesh.boundary_vertices_mask,
#     boundary_values=boundary_values,
#     f=f,
#     num_steps=10,
# )

# mesh.dump_to_JSON(
#     f"test.json",
#     {
#         "source_term": f * ~mesh.boundary_vertices_mask,
#         "poisson_equation": a,
#         "heat_equation": {"start": 0, "end": (len(u_s) - 1) * h, "data": u_s},
#         "boundary_values": boundary_values,
#     },
# )

In [128]:
mesh = Mesh(lazy=True)

### On the chart:
# compute the values of phi
# and compute the laplace beltrami operator analytically

jnp_vertices = jnp.array(mesh.vertices)

values = phi_func(jnp_vertices[:, [0, 2]])
laplace_analytical = analytical_laplace_beltrami_func(jnp_vertices[:, [0, 2]])
f = 0 * 10 * np.exp(-10 * (mesh.vertices[:, 0] ** 2 + mesh.vertices[:, 2] ** 2))

### On the manifold:
# finally map from the chart to the mesh and convert to numpy
mesh.vertices = np.array(f_func(jnp_vertices[:, [0, 2]]))
mesh.recompute()

del jnp_vertices

laplace_discrete = (mesh.laplace_matrix @ values) * ~mesh.boundary_mask


L_sps = sps.csr_matrix(mesh.laplace_matrix)
a = solve_poisson_equation(L_sps, -f, mesh.boundary_mask, boundary_values)
u_s = solve_heat_equation(
    initial_condition=np.zeros(len(mesh.vertices)) + boundary_values,
    L=L_sps,
    h=0.02,
    boundary_nodes=mesh.boundary_mask,
    boundary_values=boundary_values,
    f=f,
    num_steps=100,
)

eigenval, eigenvec = np.linalg.eig(mesh.laplace_matrix)
idx = np.argsort(eigenval)[::-1]
eigenvec = eigenvec[:, idx].T[:20]

mesh.dump_to_JSON(
    f"hyper.json",
    {
        "values": values,
        "laplace_analytical": laplace_analytical * ~mesh.boundary_mask,
        "laplace discrete": laplace_discrete,
        "source_term": f * ~mesh.boundary_mask,
        "boundary_values": boundary_values,
        "poisson_equation": a,
        "heat_equation": {
            "start": 0,
            "end": (len(u_s) - 1) * 0.02,
            "data": u_s,
        },  # u_s is a list of vertex values at each time step, of outer length len(u_s)
        "areas": np.diag(mesh.star0.toarray()) * ~mesh.boundary_mask,
        "eigvectors": {
            "start": 0,
            "end": len(eigenvec) - 1,
            "data": 5 * eigenvec,
        },
    },
)

/var/folders/vc/1d1z34l176gfm4swwfttvslr0000gn/T/ipykernel_33307/3274498193.py:58: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  -spsla.spsolve(self.star0, self.d0.T @ self.star1_circ @ self.d0)


rank of L: 686
shape of L: (686, 686)


In [129]:
mesh = Mesh.from_obj("meshes/teapot.obj")

# source_term = (mesh.vertices[:, 1] > 1).astype(float)
# source_term = mesh.vertices[:, 0]
boundary_nodes = mesh.boundary_mask.copy()

# print(np.argmax(source_term))
L_sps = sps.csr_matrix(mesh.laplace_matrix)
# source_integral = np.dot(source_term, np.diagonal(mesh.star0.toarray()))
# area = np.diagonal(mesh.star0.toarray())
# mean = source_integral / area
# source_term = source_term - mean

boundary_values = np.zeros(len(mesh.vertices))
boundary_values[0] = 0

source_term = np.zeros(len(mesh.vertices))
source_term = (mesh.vertices[:, 1] < 0.2).astype(float)
# source_term[0] = 100
# source_term[648] = 100
# source_term[1231] = 100
# source_term /= mesh.star0.diagonal()


boundary_nodes[0] = True


u_s = solve_heat_equation(
    initial_condition=np.zeros(len(mesh.vertices)),
    L=L_sps,
    h=0.002,
    boundary_nodes=mesh.boundary_mask,
    boundary_values=boundary_values * 0,
    f=source_term,
    num_steps=300,
)

stationary = solve_poisson_equation(L_sps, source_term, boundary_nodes, boundary_values)

eigenval, eigenvec = np.linalg.eig(mesh.laplace_matrix)
idx = np.argsort(eigenval)[::-1]
eigenvec = eigenvec[:, idx].T[:20]

mesh.dump_to_JSON(
    f"teapot.json",
    {
        "Source Term": source_term,
        "Poisson Equation": stationary,
        "Heat Equation": {"data": u_s, "start": 0, "end": (len(u_s) - 1) * 0.002},
        "eigvectors": {
            "start": 0,
            "end": len(eigenvec) - 1,
            "data": eigenvec,
        },
    },
)

/var/folders/vc/1d1z34l176gfm4swwfttvslr0000gn/T/ipykernel_33307/3274498193.py:58: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  -spsla.spsolve(self.star0, self.d0.T @ self.star1_circ @ self.d0)


rank of L: 1232
shape of L: (1232, 1232)
